In [4]:
import json

# JSON 파일 로드
with open("../../data/Raw_DB/romance_fantasy_only.json", "r", encoding="utf-8") as file:
    webtoon_data = json.load(file)

# 웹툰 데이터를 문자열로 변환하는 함수

def formatted_contents(contents):
    formatted = []
    for con in contents:
        formatted.append(
            {
                "제목": con.get("title", "정보 없음"),
                "타입": con.get("type", "정보 없음"),
                "플랫폼": con.get("platform", "정보 없음"),
                "장르": con.get("genre", "정보 없음"),
                "키워드": con.get("keywords", []),
                "줄거리": con.get("description", "정보 없음"),
                "연재 상태": con.get("status", "정보 없음"),
                "연령제한": con.get("age_rating", "정보 없음"),
                "가격": con.get("price_type", "정보 없음"),
                "총 회차수": con.get("episode", "정보 없음"),
                "URL": con.get("url", "정보 없음"),
            }
        )
    return formatted  # 리스트 반환

# 리스트 변환 실행
webtoon_context = formatted_contents(webtoon_data)

# ✅ 변환된 리스트 출력 (첫 3개만 예시)
for item in webtoon_context[:3]:
    print(item)

{'제목': '도망친 곳이 낙원이었다 [19세 완전판]', '타입': '웹소설', '플랫폼': '카카오페이지', '장르': '로판', '키워드': ['로맨스판타지', '회귀물', '직진남', '능력남', '도도녀', '까칠녀', '달달물', '성장물'], '줄거리': "7번의 결혼, 5번의 사별, 1번의 이혼. 외숙부의 주도로 이뤄진 정략 결혼 덕분에 아름다운 세실리아는 결혼을 거듭할수록 부유해졌다. 손에 넣은 부와 권력이 모두 제 것이라 믿었건만. 모든 것이 착각이었다! 일곱 번째 남편을 독살했다는 누명을 쓰고 처형당하는 순간에서야 그 사실을 깨달았다. 어리석었다. 외숙부를 지나치게 믿었다. 단 한 번도 제 손으로 미래를 결정한 적이 없었다. 지독한 후회와 함께 눈을 뜨자, 세실리아는 세 번째 결혼 직후로 돌아와 있었다. 세 번째 결혼 상대는 저주받은 땅 라고스의 주인, 루셀 카드로스. 그녀에게 손가락 하나 대지 않고, 2년 만의 이혼 요청에도 순순히 응해준 속을 알 수 없는 남자다. '이 남자는 결혼으로 무엇을 얻었을까?' '나는 왜 하필, 이 남자와의 결혼 직후로 돌아온 것일까?'", '연재 상태': '연재', '연령제한': '19세이용가', '가격': '정보 없음', '총 회차수': 285, 'URL': 'https://page.kakao.com/content/64688740'}
{'제목': '육아물 엄마는 꼭 죽어야 하나요?', '타입': '웹소설', '플랫폼': '카카오페이지', '장르': '로판', '키워드': '-', '줄거리': '“니에타 왕에게 명하니, 왕국의 공주를 대령하라. 그 공주와 함께 제국으로 귀환할 것이다.” 제국의 볼모가 되는 그 순간 떠오른 전생의 기억, 이곳은 황궁 육아물 속 세상이며 나는 여주인공의 엄마였다. 육아물의 정통 클리셰대로 여주인공을 낳다 죽는 바로 그 엄마. 죽기 싫어서 달아나려고 했는데, 마구간을 나서자마자 딱 걸렸다. “내 볼모가 달아나려고 했군.” 그것도 제국의 황제, 그 장본인에게 말이다. 그대로 질질 끌

In [5]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

# LLM 모델
model = ChatOpenAI(model="gpt-4o-mini")

# 프롬프트 템플릿
type_genre_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "ai",
            dedent(
                """
                    <role>
                    당신은 로맨스판타지(줄여서 로판) 웹툰과 웹소설에 대한 전문가입니다.
                    사용자가 원하는 작품 형식(웹툰 또는 웹소설)과 장르를 정확히 추출하세요.
                    장르가 없다면 로판으로 반환하십시오.
                    연재상태에 대해서 반환하십시오.
                    추가 정보는 키워드로 반환하십시오.
                    </role>
                    <genre>
                    가능한 장르 목록:
                    액션, 
                    코믹/일상, 
                    미스터리, 
                    무협, 
                    공포/스릴러, 
                    드라마, 
                    스포츠, 
                    액션/무협, 
                    스릴러, 
                    판타지, 
                    로맨스, 
                    현판, 
                    BL, 
                    로판, 
                    라이트노벨, 
                    학원/판타지, 
                    개그, 
                    무협/사극, 
                    일상
                    </genre>
                    <status>
                    월 연재,
                    화 연재,
                    수 연재,
                    목 연재,
                    금 연재, 
                    토 연재,
                    일 연재,
                    완결
                    </status>

                    반환 형식은 반드시 JSON 형식이어야 합니다.
                    <return>
                    반환 형식:
                    {{
                    "type": "웹툰" 혹은 "웹소설", / 없다면 "상관없음"
                    "genre": "장르명" / 없다면 "로판",
                    "status": "완결", "월 연재" 등의 요일별 연재 상태 / 없다면 "상관없음",
                    "age_rating": "19금", "청소년 이용불가", "성인" 등의 연령 제한 / 없다면 "상관없음",
                    "price": "무료", "기다무", "유료" 등의 가격 정보 / 없다면 "상관없음",
                    "keywords": "키워드",
                    }}
                    </return>
                    """
            ),
        ),
        ("human", "{question}"),
    ]
)

# 4. LangChain 체인 생성

parser = StrOutputParser()

type_genre_chain = type_genre_prompt | model | parser


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
print(type_genre_chain.invoke("금요일에 연재하는 웹툰 추천해줘"))

In [ ]:
recommendation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "ai",
            dedent(
                """
                <role>
                당신은 로맨스판타지(줄여서 로판) 웹소설, 웹툰의 전문가이며 로맨스판타지 장르의 남자주인공인 북부대공입니다. 당신은 모든 로맨스판타지 장르의 웹소설과 웹툰에 박식합니다.
                당신은 information에 주어진 사용자의 요구사항에 맞는 작품을 webtoon_context를 바탕으로 추천해야합니다.
                당신은 반드시 당신의 캐릭터대로 말을 해야합니다. 당신의 캐릭터는 사용자의 요구사항에 해당하지 않습니다.
                </role>

                <information>
                사용자가 요청한 정보는 다음과 같습니다:
                타입: {type}
                키워드: {keywords}
                연재 상태: {status}

                주어진 작품 데이터:
                {webtoon_context}
                </information>
                
                <check>
                적합한 작품인지 검증:
                - type:
                    사용자가 요청한 정보가 "웹툰"이면 웹툰만 추천해야 합니다.
                    사용자가 요청한 정보가 "웹소설"이면 웹소설만 추천해야 합니다.
                    사용자가 요청한 정보가 "상관없음"이면 웹툰과 웹소설 상관없이 추천해야 합니다.
                - 일치하지 않는 작품은 추천에서 제외합니다.
                - status:
                    사용자가 요청한 정보가 "연재"라면 요일이 적혀있는 경우 해당 요일에 연재하는 작품을 추천해야 합니다.
                    사용작 요청한 정보가 "연재"인데 요일이 안적혀 있다면 "완결"인 경우를 제외하고 상관없이 추천해야 합니다.
                
                추천 개수:
                - 최대 3개를 추천합니다.
                - 검증에 통과한 작품이 3개 이상일 경우: 키워드와 줄거리가 사용자의 요청 정보와 가장 가까운 3개를 추천합니다.
                - 검증에 통과한 작품이 1개 이상 3개 미만일 경우: 그대로 추천합니다.
                - 검증에 통과한 작품이 0개일 경우: 해당하는 작품이 없다고 답변하십시오.
                </check>
                
                <result>
                반환형식:
                {{
                    "title": 제목,
                    "type": 타입,
                    "platform": 플랫폼,
                    "status": 연재상태,
                    "keyword": 키워드,
                    "description": 줄거리 요약,
                    "reason": 추천 이유,
                    "url": url
                }}
                </result>
                """
            ),
        ),
        ("human", "{question}"),
    ]
)

recommendation_chain = recommendation_prompt | model | parser



In [ ]:
print(
    recommendation_chain.invoke(
        {"type": "웹툰", "keywords": "상관없음", "webtoon_context": webtoon_context,"status": "금 연재",
        "question":"금요일에 연재하는 웹툰 추천해줘."}
    )
)

In [ ]:
from langchain.schema.runnable import RunnableParallel
import json

complete_chain = RunnableParallel(
    {
        "parsed_result": type_genre_chain | (lambda x: json.loads(x) if isinstance(x, str) else x),
        "question": lambda x: x,  # 사용자의 원래 질문 전달
    }
) | (
    lambda inputs: {
        "type": inputs["parsed_result"].get("type", "상관없음"),
        "genre": inputs["parsed_result"].get("genre", "로판"),
        "keywords": inputs["parsed_result"].get("keywords", []),
        "webtoon_context": webtoon_context,
        "question": inputs["question"],  # ✅ 사용자의 원래 질문을 그대로 전달
    }
) | recommendation_chain
